In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('regression').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [48]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")


In [34]:
output = assembler.transform(data)

In [66]:
output.select("features").show()
# output.describe().show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [50]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            Features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [51]:
final_data = output.select("features",'Yearly Amount Spent')

In [52]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [53]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                362|
|   mean|  498.6737418754012|
| stddev|  77.08148718929856|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



In [54]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                138|
|   mean|  500.9936563072479|
| stddev|  85.16566633638799|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [41]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                137|
|   mean|  500.6894229675171|
| stddev|  84.08863254781633|
|    min|  282.4712457199145|
|    max|  765.5184619388373|
+-------+-------------------+



In [55]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [58]:
lr_Model = lr.fit(train_data)

In [59]:
test_results = lr_Model.evaluate(test_data)

In [60]:
test_results.rootMeanSquaredError

9.936116039044713

In [61]:
test_results.r2

0.986289202033019

In [82]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [86]:
unlabeled_data = test_data.select('features')

In [89]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.8162006488763...|
|[31.0472221394875...|
|[31.1695067987115...|
|[31.3662121671876...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.8745516945853...|
|[31.9120759292006...|
|[31.9549038566348...|
|[31.9563005605233...|
|[32.0047530203648...|
|[32.0085045178551...|
|[32.0498393904573...|
|[32.0789475795693...|
+--------------------+
only showing top 20 rows



In [90]:
predictions = lr_Model.transform(unlabeled_data)